<a href="https://colab.research.google.com/github/parshwa1999/Map-Segmentation/blob/master/PSPNet_RoadMapSeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Semantic Segmentation of Aerial Images
###Road Training

####Mounting Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
base_path = "gdrive/My\ Drive/MapSegClean/"
%cd gdrive/My\ Drive/MapSegClean/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/
/content/gdrive/My Drive/MapSegClean


####Importing Libraries

In [0]:
import cv2
import numpy as np
import h5py

In [3]:
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [0]:
from keras.models import Model, load_model
import tensorflow as tf
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras import optimizers
from keras.layers import BatchNormalization
from tensorflow.keras.metrics import MeanIoU
import keras

from skimage import color
import pickle

####Read Data From h5 files

In [0]:
h5f = h5py.File('Cpy_clean_roadlabel.h5','r')
roadlabel = h5f['clean_roadlabel'][:]
h5f.close()

h5f = h5py.File('Cpy_clean_road.h5','r')
road = h5f['clean_road'][:]
h5f.close()

In [6]:
#for i in range(road.shape[0]):
#  cv2.imwrite("Road/" + str(i) + ".png", road[i])

#for i in range(roadlabel.shape[0]):
#  cv2.imwrite("RoadLabel/" + str(i) + ".png", roadlabel[i])
pip install -U segmentation-models

####Loss Function

In [0]:
def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou

def dice_coef(y_true, y_pred, smooth = 1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def soft_dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)


####Preprocecssing Data

In [8]:
print(roadlabel.shape)

roadlabelgray = []

for i in range(roadlabel.shape[0]):
  roadlabelgray.append(cv2.cvtColor(roadlabel[i], cv2.COLOR_BGR2GRAY))

roadlabelgray = np.asarray(roadlabelgray)

roadlabel = np.expand_dims(roadlabelgray, -1)

del roadlabelgray

print(roadlabel.shape)

(22911, 256, 256, 3)
(22911, 256, 256, 1)


In [9]:
print(roadlabel[:, 8:256-8, 8:256-8].shape)

(22911, 240, 240, 1)


In [0]:
seed = 56
IMAGE_HEIGHT = IMAGE_WIDTH = 256
NUM_CHANNELS = 3

####Model

In [11]:
from segmentation_models import PSPNet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score

Segmentation Models: using `keras` framework.


In [12]:
model = PSPNet(input_shape=(240, 240, 3), encoder_weights=None, classes=1, psp_use_batchnorm=True, psp_pooling_type='max', activation='sigmoid')
#model = Unet(input_shape=(256, 256, 3), weights=None, activation='elu')
model.summary()














Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 240, 240, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 240, 240, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 240, 240, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 120, 120, 64) 0           block1_conv2[0][0]               
_______________________________________________________________________________

####Hyperparameters


In [0]:
LEARNING_RATE = 0.0001
BATCH_SIZE = 32

####Path to save Model

In [0]:
model_path = "Models/Final_PSPNet_road_weights.h5"

In [15]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                          road:  4.2 GiB
                         Model:  2.0 KiB
                        Conv2D:  2.0 KiB
            BatchNormalization:  2.0 KiB
                  MaxPooling2D:  1.4 KiB
                       Dropout:  1.0 KiB
                        Lambda:  1.0 KiB
               Conv2DTranspose:  1.0 KiB
                       MeanIoU:  1.0 KiB
                           _i7:  615.0 B


####Callbacks

In [16]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://f19c2793.ngrok.io


In [0]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from datetime import datetime

checkpointer = ModelCheckpoint(model_path,
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

In [0]:
earlystopper = EarlyStopping(monitor = 'val_loss', 
                             min_delta = 0, 
                             patience = 7,
                             mode='auto',
                            restore_best_weights = True)

In [19]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4)

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1335: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


####Compile Model

In [20]:
opt = keras.optimizers.adam(LEARNING_RATE)
model.compile(
      optimizer=opt,
      loss=soft_dice_loss,
      metrics=[iou_coef])   

In [0]:
history = model.fit(road[:, 8:256-8, 8:256-8],
                    (roadlabel[:, 8:256-8, 8:256-8].astype(np.uint8)/255).astype(np.uint8),
                    validation_split = 0.1,
                    epochs=100,
                    batch_size = 32,
                    callbacks = [checkpointer, earlystopper, lr_reducer, TensorBoardColabCallback(tbc)]
                       )

with open("History/PSPNet_Road_History.pickle", 'wb') as f:
    pickle.dump(history, f)

Train on 20619 samples, validate on 2292 samples
Epoch 1/100
20619/20619 [==============================] - 824s 40ms/step - loss: 0.4195 - iou_coef: 0.3520 - val_loss: 0.4169 - val_iou_coef: 0.4025

Epoch 00001: val_loss improved from inf to 0.41689, saving model to Models/Final_PSPNet_road_weights.h5

Epoch 2/100
20619/20619 [==============================] - 809s 39ms/step - loss: 0.3289 - iou_coef: 0.4790 - val_loss: 0.3187 - val_iou_coef: 0.4842

Epoch 00002: val_loss improved from 0.41689 to 0.31869, saving model to Models/Final_PSPNet_road_weights.h5
Epoch 3/100
20619/20619 [==============================] - 809s 39ms/step - loss: 0.3080 - iou_coef: 0.5242 - val_loss: 0.3073 - val_iou_coef: 0.5421

Epoch 00003: val_loss improved from 0.31869 to 0.30732, saving model to Models/Final_PSPNet_road_weights.h5
Epoch 4/100
20619/20619 [==============================] - 810s 39ms/step - loss: 0.2944 - iou_coef: 0.5451 - val_loss: 0.3071 - val_iou_coef: 0.5212

Epoch 00004: val_loss impr

In [0]:
#import pickle
#with open("History/Road_History.pickle", 'rb') as f:
#    history = pickle.load(f)

In [0]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['iou_coef'])
plt.plot(history.history['val_iou_coef'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [21]:
model.load_weights(model_path)

history = model.fit(road[:, 8:256-8, 8:256-8],
                    (roadlabel[:, 8:256-8, 8:256-8].astype(np.uint8)/255).astype(np.uint8),
                    validation_split = 0.1,
                    epochs=100,
                    batch_size = 32,
                    callbacks = [checkpointer, earlystopper, lr_reducer, TensorBoardColabCallback(tbc)]
                       )

with open("History/PSPNet_Road_History_1.pickle", 'wb') as f:
    pickle.dump(history, f)



Train on 20619 samples, validate on 2292 samples


Epoch 1/100
20619/20619 [==============================] - 221s 11ms/step - loss: 0.2304 - iou_coef: 0.6512 - val_loss: 0.2681 - val_iou_coef: 0.6050

Epoch 00001: val_loss improved from inf to 0.26814, saving model to Models/Final_PSPNet_road_weights.h5

Epoch 2/100
20619/20619 [==============================] - 210s 10ms/step - loss: 0.2252 - iou_coef: 0.6608 - val_loss: 0.3049 - val_iou_coef: 0.5751

Epoch 00002: val_loss did not improve from 0.26814
Epoch 3/100
20619/20619 [==============================] - 210s 10ms/step - loss: 0.2208 - iou_coef: 0.6639 - val_loss: 0.2723 - val_iou_coef: 0.6049

Epoch 00003: val_loss did not improve from 0.26814
Epoch 4/100
20619/20619 [==============================] - 210s 10ms/step - loss: 0.2171 - iou_coef: 0.6745 - val_loss: 0.2632 - val_iou_coef: 0.6104

Epoch 00004: val_loss improved from 0.26814 to 0.26317, saving model to Models/Final_PSPNet_road_weights.h5
Epoch 5/100
20619/20619 [===